---
---
# TorchScript Modeli ile Tek Görüntü Üzerinde C++/LibTorch Inference Uygulaması

----
---

## Neyi iyileştirdim?


* Daha derin bir CNN mimarisi kullandım (256 kanala kadar çıkıyor).

* Her Conv katmanından sonra BatchNorm + ReLU ekledim.

* MaxPool + AdaptiveAvgPool2d ile feature map’leri sıkıştırdım.

* Dev gibi 128*16*16 dense yerine Global Average Pooling + daha küçük bir classifier kullandım.

* Dropout ekleyerek overfitting’i azaltmaya çalıştım.

* CIFAR10 için Normalize ekledim (mean/std).

* Kod yapısını biraz daha temizledim (modeli iki kere yaratma gibi ufak bug’ları düzelttim).

---
---

# Yeni model + eğitim + TorchScript export (tam script)(Python)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# --------------------------
# Geliştirilmiş CNN Mimarisi
# --------------------------

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)


class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ImprovedCNN, self).__init__()

        self.features = nn.Sequential(
            # Giriş: [B, 3, 64, 64]
            ConvBlock(3, 32),              # [B, 32, 64, 64]
            ConvBlock(32, 64),             # [B, 64, 64, 64]
            nn.MaxPool2d(2, 2),            # [B, 64, 32, 32]

            ConvBlock(64, 128),            # [B, 128, 32, 32]
            ConvBlock(128, 128),           # [B, 128, 32, 32]
            nn.MaxPool2d(2, 2),            # [B, 128, 16, 16]

            ConvBlock(128, 256),           # [B, 256, 16, 16]
            nn.MaxPool2d(2, 2),            # [B, 256, 8, 8]

            # Global Average Pooling: [B, 256, 1, 1]
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                  # [B, 256]
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)    # [B, num_classes]
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [2]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # 0-1
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    ),
])

train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

print("Dataset hazır, model oluşturuluyor...")


Dataset hazır, model oluşturuluyor...


In [3]:
# --------------------------
# Model, cihaz, loss, optimizer
# --------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanilan cihaz:", device)

model = ImprovedCNN(num_classes=10).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


Kullanilan cihaz: cpu
ImprovedCNN(
  (features): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (4): ConvBlock(
      (block): Sequential(
        (0):

In [4]:
EPOCHS = 1
best_acc= 0.0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0.0
    total = 0.0

    for images , labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs,labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _,preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images , labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _ , preds = outputs.max(1)
            correct_test += preds.eq(labels).sum().item()
            total_test += labels.size(0)
        
    test_acc = 100 * correct_test / total_test
    print(f"Epoch [{epoch+1}/{EPOCHS}] | "
          f"Train Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.2f}% | "
          f"Test Acc: {test_acc:.2f}%")
    
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict() , "best_model.pth")
        print(f"  -> Yeni en iyi model kaydedildi! (best_model.pth, acc={best_acc:.2f}%)")

print("Egitim tamamlandi. En iyi test accuracy:", best_acc)

Epoch [1/1] | Train Loss: 1.5548 | Train Acc: 41.82% | Test Acc: 51.72%
  -> Yeni en iyi model kaydedildi! (best_model.pth, acc=51.72%)
Egitim tamamlandi. En iyi test accuracy: 51.722


In [5]:
export_model = ImprovedCNN(num_classes=10)
export_model.load_state_dict(torch.load("best_model.pth",map_location ="cpu"))
export_model.eval()

example_input = torch.randn(1,3,64,64)

traces_script_module = torch.jit.trace(export_model , example_input)
traces_script_module.save("model_ts.pt")
print("TorchScript modeli kaydedildi: model_ts.pt")

TorchScript modeli kaydedildi: model_ts.pt


## Aşağıdaki hücre C++'ın libtorch modelidir.

```cpp
// ---------------------------------------------------------
// LibTorch + OpenCV ile TorchScript model inference örneği
// (ImprovedCNN + CIFAR10 Normalize)
// ---------------------------------------------------------
// Bu dosyada:
//  - TorchScript modeli (model_ts.pt) yüklüyorum
//  - OpenCV ile bir görüntü (test.jpg) okuyorum
//  - cv::Mat -> torch::Tensor (BGR -> RGB, resize, normalize) yapıyorum
//  - module.forward() ile tahmin alıyorum
//  - softmax + argmax ile sınıf index'ini hesaplıyorum
// ---------------------------------------------------------

#include <iostream>
#include <fstream>
#include <string>
#include <vector>

#include <torch/script.h>     // TorchScript API
#include <opencv2/opencv.hpp> // OpenCV ana header

// ---------------------------------------------------------
// CIFAR-10 Normalize değerleri (PyTorch tarafındaki ile aynı)
// transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
//                      std=[0.2470, 0.2435, 0.2616])
// ---------------------------------------------------------
const float CIFAR10_MEAN[3] = {0.4914f, 0.4822f, 0.4465f}; // R, G, B
const float CIFAR10_STD[3]  = {0.2470f, 0.2435f, 0.2616f};

// ---------------------------------------------------------
// cv::Mat (BGR) -> torch::Tensor [1, 3, H, W]
// ---------------------------------------------------------
// Adımlar:
//  1) Resize (H x W)
//  2) BGR -> RGB
//  3) uint8 -> float32, 0-1 normalize
//  4) Kanal bazlı Normalize: (x - mean) / std (RGB)
//  5) HWC -> NHWC -> NCHW (PyTorch formatı)
//  6) contiguous() + clone() ile bellek güvence altına alma
// ---------------------------------------------------------
torch::Tensor preprocess_image(const cv::Mat &img_bgr, int target_size)
{
    cv::Mat img;

    // Orijinal görüntüyü kopyala (orijinale dokunmamak için)
    img_bgr.copyTo(img);

    // 1) Resize (W x H)
    cv::resize(img, img, cv::Size(target_size, target_size));

    // 2) BGR -> RGB (OpenCV BGR kullanır, model RGB bekliyor)
    cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

    // 3) uint8 -> float32 + normalize 0-1
    cv::Mat img_float;
    img.convertTo(img_float, CV_32F, 1.0f / 255.0f); // [0,1] aralığı

    // 4) Kanal bazlı normalize (R,G,B)
    //    PyTorch'taki transforms.Normalize ile aynı işlem:
    //    x = (x - mean) / std
    {
        std::vector<cv::Mat> channels(3);
        cv::split(img_float, channels); // channels[0]=R, [1]=G, [2]=B (artık RGB)

        for (int c = 0; c < 3; ++c)
        {
            // channels[c] = (channels[c] - mean[c]) / std[c];
            channels[c] = (channels[c] - CIFAR10_MEAN[c]) / CIFAR10_STD[c];
        }

        cv::merge(channels, img_float);
    }

    // 5) HWC -> NHWC formatında tensöre çevir
    auto tensor_image = torch::from_blob(
        img_float.data,
        {1, img_float.rows, img_float.cols, 3}, // [N, H, W, C]
        torch::TensorOptions().dtype(torch::kFloat32));

    // 6) NHWC -> NCHW (PyTorch standardı)
    tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N, C, H, W]

    // contiguous + clone: OpenCV belleğinden ayrıştırmak için
    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}

// ---------------------------------------------------------
// TorchScript module üzerinde inference fonksiyonu
// ---------------------------------------------------------
torch::Tensor run_inference(torch::jit::script::Module &module,
                            const torch::Tensor &input)
{
    // TorchScript forward() IValue listesi bekler
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input);

    // module.forward() -> IValue, toTensor() ile tensöre çevir
    torch::Tensor output = module.forward(inputs).toTensor();
    return output;
}

int main()
{
    try
    {
        std::cout << "[INFO] Program basladi.\n";

        // ------------------------------
        // 1) Model dosya yolu
        // ------------------------------
        const std::string model_path = "model_ts.pt";

        // Dosya gerçekten var mı kontrol et
        {
            std::ifstream f(model_path);
            if (!f.good())
            {
                std::cerr << "[HATA] model_ts.pt bulunamadi!\n";
                std::cerr << "       model_ts.pt dosyasini exe ile ayni klasore koy.\n";
                std::cout << "Enter'a basip cikabilirsin...\n";
                std::cin.get();
                return -1;
            }
        }

        // ------------------------------
        // 2) TorchScript modeli yükle
        // ------------------------------
        std::cout << "[INFO] Model yukleniyor...\n";
        torch::jit::script::Module module = torch::jit::load(model_path);

        // CPU kullanıyoruz (GPU varsa kCUDA da kullanılabilir)
        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yuklendi ve eval modunda.\n";

        // ------------------------------
        // 3) Test resmi oku
        // ------------------------------
        const std::string image_path = "test.jpg"; // exe ile aynı klasörde olmalı
        cv::Mat img_bgr = cv::imread(image_path);

        if (img_bgr.empty())
        {
            std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
            std::cerr << "       test.jpg dosyasini exe ile ayni klasore koy.\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }

        std::cout << "[OK] Resim yuklendi. Boyut: "
                  << img_bgr.cols << "x" << img_bgr.rows << " (W x H)\n";

        // ------------------------------
        // 4) Preprocess -> Tensor
        // ------------------------------
        int target_size = 64; // Python tarafında Resize((64,64)) kullandık
        torch::Tensor input_tensor = preprocess_image(img_bgr, target_size);

        std::cout << "[DEBUG] Input tensor shape: " << input_tensor.sizes() << "\n";

        // Beklenen shape: [1, 3, 64, 64]
        auto sizes = input_tensor.sizes();
        if (!(sizes.size() == 4 &&
              sizes[0] == 1 &&
              sizes[1] == 3 &&
              sizes[2] == target_size &&
              sizes[3] == target_size))
        {
            std::cerr << "[HATA] Input tensor beklenenden farkli! Gelen: "
                      << sizes << "\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }

        // ------------------------------
        // 5) Inference (forward)
        // ------------------------------
        std::cout << "[INFO] Inference calistiriliyor...\n";
        torch::Tensor output = run_inference(module, input_tensor);

        std::cout << "[DEBUG] Output tensor shape: " << output.sizes() << "\n";

        // ------------------------------
        // 6) Postprocess: softmax + argmax
        // ------------------------------
        // Çıkış tensörü: [1, num_classes] (örn. [1, 10])
        torch::Tensor probs = torch::softmax(output, 1); // sınıf olasılıkları
        torch::Tensor pred_class = probs.argmax(1);      // en yüksek olasılıklı sınıf index'i

        std::cout << "Output (logits): " << output << "\n";
        std::cout << "Predicted class index: "
                  << pred_class.item<int>() << "\n";

        std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error &e)
    {
        std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception &e)
    {
        std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}


----
---


# Gelin bu main.cpp ı inceleyelim.Adım adım anlatarak gidelim
---
---

# 1️⃣ Header’lar ve global sabitler

```cpp
#include <iostream>
#include <fstream>
#include <string>
#include <vector>

#include <torch/script.h>     // TorchScript API
#include <opencv2/opencv.hpp> // OpenCV ana header
```

Burada :

* C++ tarafında giriş/çıkış (iostream), dosya kontrol (fstream), string, vector gibi temel şeyleri dahil ediyorum.

* torch/script.h ile TorchScript modelini C++’tan kullanmaya açıyorum.

* opencv2/opencv.hpp ile tüm temel OpenCV fonksiyonlarına erişiyorum.

```cpp
const float CIFAR10_MEAN[3] = {0.4914f, 0.4822f, 0.4465f}; // R, G, B
const float CIFAR10_STD[3]  = {0.2470f, 0.2435f, 0.2616f};
```
Burada ben:

* Python tarafında transforms.Normalize(mean=[...], std=[...]) ile kullandığım CIFAR-10 normalize değerlerini C++ tarafında da sabitliyorum.

* Sıra: [R, G, B].

Yani:

**“Ben eğitimde hangi normalize değerlerini kullandıysam, inference’ta da birebir aynısını C++’ta uyguluyorum ki model tutarlı çalışsın.”**

---

# 2️⃣ preprocess_image fonksiyonu
```cpp
torch::Tensor preprocess_image(const cv::Mat &img_bgr, int target_size)
{
    cv::Mat img;

    // Orijinal görüntüyü kopyala (orijinale dokunmamak için)
    img_bgr.copyTo(img);
```
Burada :

* Fonksiyonun içindeki img ile oynayayım, dışarıdaki img_bgr bozulmasın diye önce kopya alıyorum.

## 2.1 Resize
```cpp
    cv::resize(img, img, cv::Size(target_size, target_size));
```


Burada:

* “Ben resmi (target_size, target_size) yani 64×64’e kırpıyorum/ölçekliyorum. Python tarafında da Resize((64,64)) kullandığım için birebir aynı boyutu sağlıyorum.”

## 2.2 BGR → RGB
```cpp
    cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
```


Burada :

* OpenCV’nin default BGR formatını, model tarafının beklediği RGB’ye çeviriyorum.

Yapmasam:

* Kanal sırası ters olur, model B kanalını R gibi görür; renk uzayı bozulur, accuracy çöker.

## 2.3 uint8 → float32 + [0,1] normalize
```cpp   
    cv::Mat img_float;
    img.convertTo(img_float, CV_32F, 1.0f / 255.0f); // [0,1] aralığı
```


Burada :

* Piksel değerlerini 0–255 uint8’den 0–1 float32 aralığına indiriyorum.

* PyTorch’ta ToTensor() da bunu yapıyordu; aynısını C++’ta tekrarlıyorum.

Yapmasam:

* Model eğitimde float32 [0,1] veriye alışmış, C++’ta 0–255 verirsem dağılım tamamen değişir, model saçmalar.

## 2.4 Kanal bazlı normalize (en kritik yeni kısım)
```cpp
    {
        std::vector<cv::Mat> channels(3);
        cv::split(img_float, channels); // channels[0]=R, [1]=G, [2]=B (artık RGB)

        for (int c = 0; c < 3; ++c)
        {
            // channels[c] = (channels[c] - mean[c]) / std[c];
            channels[c] = (channels[c] - CIFAR10_MEAN[c]) / CIFAR10_STD[c];
        }

        cv::merge(channels, img_float);
    }
```
Burada şunu yapıyorum:

**cv::split ile resmi 3 kanala ayırıyorum:**

* channels[0] → R

* channels[1] → G

* channels[2] → B

**Döngü içinde her kanal için:**

* channels[c] = (channels[c] - mean[c]) / std[c];

* Yani transforms.Normalize’in birebir aynısını C++ koduna döküyorum.

* cv::merge ile tekrar 3 kanalı birleştirip img_float haline getiriyorum.

**Yapmasam ne olur?**

* Eğitimde normalize edilmiş dağılıma göre öğrenmiş model, inference’ta normalize edilmemiş veri görür.

* Bu, modelin “gördüğü input distribution”u değiştirir → test accuracy ciddi düşer.

* Özellikle batchnorm + daha derin modelde bu çok daha kritik hale gelir.

**Özet cümle:**

* “Ben burada resmi kanal bazında normalize ederek, C++ tarafında da PyTorch’taki Normalize’i birebir taklit ediyorum; böylece eğitim–inference dağılımı tutarlı kalıyor.”


## 2.5 from_blob ile tensöre sarma
```cpp   
    auto tensor_image = torch::from_blob(
        img_float.data,
        {1, img_float.rows, img_float.cols, 3}, // [N, H, W, C]
        torch::TensorOptions().dtype(torch::kFloat32));
```


Burada :

* img_float.data RAM’ine dokunmadan,

* Ona [1, H, W, 3] şeklinde NHWC formatında bir tensör anlamı yüklüyorum.

* dtype’ı float32 olarak net belirtiyorum.

* Bunu zaten birlikte detaylı inceledik: shape/dtype yanlış yazarsam:

* Modelin shape’i ile uyumsuzluk,

* Veya veri tipinin bozulması gibi sıkıntılar çıkar.

## 2.6 NHWC → NCHW + contiguous + clone
```cpp   
    tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N, C, H, W]

    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}
```


Burada :

* permute ile eksenleri [N,H,W,C]’den [N,C,H,W]’ye çeviriyorum.

* contiguous() ile bellek düzenini standart hale getiriyorum.

* clone() ile OpenCV RAM’inden ayrılıp kendi sahibi olduğum bir tensör kopyası alıyorum.

---

# 3️⃣ run_inference fonksiyonu
```cpp
torch::Tensor run_inference(torch::jit::script::Module &module,
                            const torch::Tensor &input)
{
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input);

    torch::Tensor output = module.forward(inputs).toTensor();
    return output;
}
```


Burada :

* Modeli referans ile alıyorum (kopyalamıyorum).

* Input tensörü bir IValue listesine sarıyorum (TorchScript forward API’si böyle istiyor).

* module.forward(inputs) sonucunu toTensor() ile tekrar tensöre çeviriyorum.
---
# 4️⃣ main – Genel akış
## 4.1 try–catch ve başlangıç logu
```cpp
int main()
{
    try
    {
        std::cout << "[INFO] Program basladi.\n";
```


Burada :

* Tüm inference pipeline’ını try içine alıyorum.

* Logla program başladığını net yazıyorum.

## 4.2 Model yolunu belirleme ve dosya kontrolü
```cpp   
        const std::string model_path = "model_ts.pt";

        {
            std::ifstream f(model_path);
            if (!f.good())
            {
                std::cerr << "[HATA] model_ts.pt bulunamadi!\n";
                ...
                return -1;
            }
        }
```

Burada :

* model_ts.pt dosyasının gerçekten var olup olmadığını önceden kontrol ediyorum.

* Yoksa LibTorch içinde karmaşık hata almak yerine kendim düzgün mesaj basıyorum.


## 4.3 TorchScript modeli yükleme
```cpp    
        std::cout << "[INFO] Model yukleniyor...\n";
        torch::jit::script::Module module = torch::jit::load(model_path);

        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yuklendi ve eval modunda.\n";
```


Burada :

* TorchScript modelini dosyadan yüklüyorum.

* CPU’ya alıyorum (kCUDA da kullanabilirdim).

* eval() ile inference moduna çekiyorum (dropout vs. devre dışı).

**Bunu yapmazsam:**

* Training davranışları devam eder, sonuçlar tutarsızlaşır.

## 4.4 Test görüntüsünü yükleme
```cpp
        const std::string image_path = "test.jpg";
        cv::Mat img_bgr = cv::imread(image_path);

        if (img_bgr.empty())
        {
            std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
            ...
            return -1;
        }

        std::cout << "[OK] Resim yuklendi. Boyut: "
                  << img_bgr.cols << "x" << img_bgr.rows << " (W x H)\n";
```


Burada :

* test.jpg’i yüklüyorum.

* Yüklenemediyse net hata mesajı veriyorum.

* Yüklenirse boyutları logluyorum.

## 4.5 Preprocess → Tensor
```cpp   
        int target_size = 64;
        torch::Tensor input_tensor = preprocess_image(img_bgr, target_size);

        std::cout << "[DEBUG] Input tensor shape: " << input_tensor.sizes() << "\n";

        auto sizes = input_tensor.sizes();
        if (!(sizes.size() == 4 &&
              sizes[0] == 1 &&
              sizes[1] == 3 &&
              sizes[2] == target_size &&
              sizes[3] == target_size))
        {
            std::cerr << "[HATA] Input tensor beklenenden farkli! Gelen: "
                      << sizes << "\n";
            ...
            return -1;
        }
```


Burada :

* Az önce incelediğimiz preprocess_image’i çağırıp, tam modele uygun input tensörü üretiyorum.

Sonra boyutları kontrol ediyorum:

* 4 boyutlu mu?

* Batch=1, kanal=3 mü?

* 64×64 mi?

**Yapmazsam:***

* Yanlış shape modelin içinde patlar, debug zorlaşır.

* Ben bu şekilde “şekil garantisini” main düzeyinde sağlamış oluyorum.


## 4.6 Inference çağrısı
```cpp   
        std::cout << "[INFO] Inference calistiriliyor...\n";
        torch::Tensor output = run_inference(module, input_tensor);

        std::cout << "[DEBUG] Output tensor shape: " << output.sizes() << "\n";
```


Burada :

* run_inference ile modeli input tensörle koşturuyorum.

* Dönen output tensörün shape’ini logluyorum (genelde [1,10]).


## 4.7 Postprocess: softmax + argmax + item
```cpp
        torch::Tensor probs = torch::softmax(output, 1); // sınıf olasılıkları
        torch::Tensor pred_class = probs.argmax(1);      // en yüksek olasılıklı sınıf index'i

        std::cout << "Output (logits): " << output << "\n";
        std::cout << "Predicted class index: "
                  << pred_class.item<int>() << "\n";
```


Burada :

* output’u 1. boyut (class boyutu) üzerinden softmax’la olasılığa çeviriyorum.

* argmax(1) ile en yüksek olasılıklı sınıf indeksini alıyorum.

* Sonra bunu item<int>() ile düz C++ int’e çevirip ekrana yazıyorum.

## 4.8 Program sonu + catch blokları
```cpp
        std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error &e)
    {
        ...
    }
    catch (const std::exception &e)
    {
        ...
    }

    return 0;
}
```


Burada :

* Program sonunda Enter bekleyerek konsolun hemen kapanmasını engelliyorum.

* LibTorch özel hataları (c10::Error) ve genel C++ hatalarını (std::exception) ayrı ayrı yakalayıp logluyorum.
---
# Özet – Bu main.cpp’yi şöyle tarif edelim... :

**“Ben bu main.cpp’de PyTorch’ta eğittiğim ImprovedCNN modelini TorchScript olarak C++’a taşıyorum, C++ tarafında OpenCV ile resmi okuyup CIFAR-10’da kullandığım normalize işlemlerini birebir kopyalıyorum, resmi tensöre çevirip modeli koşturuyorum ve en sonunda tahmin edilen sınıf indeksini hesaplayıp ekrana yazdırıyorum; tüm süreci try–catch ve loglarla kontrollü ve okunabilir hale getiriyorum.”**

---

# 📁 Proje Klasör Yapısı ve Build Süreci


## 1. Proje Kök Dizini

C++ tarafında çalıştırdığım proje aşağıdaki yapıya sahip olmalıdır:
```bash
Uygulama-6/
│
├── CMakeLists.txt        # CMake yapılandırma dosyası
├── main.cpp              # C++ kaynak dosyası
│
└── build/                # Build klasörü (CMake tarafından oluşturulur)
    ├── Debug/            # Debug build çıktıları
    └── Release/          # Release build çıktıları (KULLANDIĞIMIZ)
```


* Çalıştırılacak dosyalar Release klasöründe bulunur.

## 2. Release Klasöründe Olması Gereken Dosyalar

Modeli başarıyla çalıştırmak için exe ile aynı klasörde aşağıdaki dosyalar bulunmalıdır:
```bash
build/Release/
│
├── improved_inference.exe     # Derlenen çalıştırılabilir dosya
├── model_ts.pt                # TorchScript modeli
├── test.jpg                   # Sınıflandırılacak resim
│
├── *.dll                      # LibTorch ve OpenCV DLL dosyaları
│                             # (CMake ile otomatik kopyalanabilir)
└── diğer CMake çıktı dosyaları
```


#### **ÖNEMLİ KURAL:**

>Çalıştırdığım .exe hangi klasörde ise, model_ts.pt ve test.jpg aynı klasörde bulunmalıdır.

>DLL dosyaları da aynı klasörde bulunmalıdır.

## 3. Build İşlemi
### 3.1 Build Klasörü Oluşturma

Terminalde proje kök dizinine gidip:
```bash
mkdir build
cd build
cmake ..
```


* Bu komut, proje yapılandırmasını oluşturur.

### 3.2 Release Modunda Derleme
```bash
cmake --build . --config Release
```


Derleme tamamlandığında çalıştırılabilir dosya buraya oluşur:

**build/Release/improved_inference.exe**

## 4. Programı Çalıştırma

Çalıştırmak için terminalden Release klasörüne geçilir:
```bash
cd build/Release
./improved_inference.exe
```


Program artık:

* model_ts.pt dosyasını yükleyebilir

* test.jpg dosyasını okuyabilir

* inference yapabilir

## 5. Özet

* main.cpp ve CMakeLists.txt proje kökünde durur.

* cmake .. ve cmake --build . --config Release komutlarıyla build alınır.

* Exe, model_ts.pt ve test.jpg’nin aynı klasörde olması zorunludur.

* DLL dosyaları da exe ile aynı klasörde olmalıdır.

----